In [8]:
"""
1) Query DB:
SELECT 
    OBJECT_NAME(f.parent_object_id) AS TableName,
	OBJECT_NAME (f.referenced_object_id) AS ReferenceTableName,
    COL_NAME(fc.parent_object_id, fc.parent_column_id) AS ForeignKeyColumnName
FROM 
    sys.foreign_keys AS f
INNER JOIN 
    sys.foreign_key_columns AS fc 
ON 
    f.OBJECT_ID = fc.constraint_object_id

2) ChatGPT:
Converti la seguente tabella in una serie di tuple.
Per esempio: 
("a", "b", "c"),

.................................................

3) inserisci le tuple nel codice ed esegui
"""

import networkx as nx
import matplotlib.pyplot as plt
from ipywidgets import interact, SelectMultiple, Output, HBox, Checkbox

# Inserisci qui le tue relazioni
relazioni = [
    ("ApplicationRolePermission", "ApplicationRole", "RoleID"),
    ("ApplicationRoleUser", "ApplicationRole", "RoleID"),
    ("RoleRequest", "ApplicationRole", "RoleID"),
    ("ApplicationRoleApprover", "ApplicationRole", "RoleID"),
    ("Device", "DeviceFamily", "DeviceFamilyId"),
    ("DeviceRole_User", "DeviceRole", "RoleID"),
    ("PhysicalDevice_User", "DeviceRole", "DeviceRoleID"),
    ("DeviceRolePermission", "DeviceRole", "RoleId"),
    ("ObjectId", "ObjectIdsRequest", "OidRequestID"),
    ("PhysicalDevice_User", "PhysicalDevice", "PhysicalDeviceID"),
    ("Device_DeviceCategory", "DeviceCategory", "DeviceCategoryID"),
    ("RoleRequestApproval", "RoleRequest", "RoleRequestID"),
    ("DeviceRole", "Device", "DeviceID"),
    ("PhysicalDevice", "Device", "DeviceID"),
    ("Device_DeviceCategory", "Device", "DeviceID"),
    ("DevicePermission", "Device", "DeviceID"),
    ("ApplicationPropertyVal", "ApplicationRoleUser", "RoleUserID"),
    ("File", "FileType", "TypeID"),
    ("NotificationTarget", "Notification", "NotificationID"),
    ("FeatureAccess", "Feature", "FeatureID"),
    ("DeviceRolePermission", "DevicePermission", "PermissionId"),
    ("Notification", "NotificationTopic", "TopicID"),
    ("ReportDownload", "Report", "ReportID"),
    ("ApplicationPropertyVal", "ApplicationPermissionUser", "PermissionUserID"),
    ("ApplicationPropertyVal", "ApplicationRolePermission", "RolePermissionID"),
    ("ApplicationPropertyVal", "ApplicationPropertyDef", "ApplicationID"),
    ("ApplicationPropertyVal", "ApplicationPropertyDef", "Name"),
    ("FileSharing", "File", "FileID"),
    ("FileVersion", "File", "FileID"),
    ("UserGroup", "Group", "GroupID"),
    ("FileSharing", "Group", "GroupID"),
    ("RoleRequestApproval", "User", "ForwardedBy"),
    ("RoleRequestApproval", "User", "CompletedBy"),
    ("ApplicationRoleApprover", "User", "UserID"),
    ("PhysicalDevice_User", "User", "OwnerID"),
    ("PhysicalDevice_User", "User", "UserID"),
    ("RoleRequest", "User", "UserID"),
    ("RoleRequest", "User", "CompletedBy"),
    ("DeviceRole_User", "User", "UserID"),
    ("File", "User", "Owner"),
    ("Relt", "User", "LockedBy"),
    ("Relt", "User", "UnlockedBy"),
    ("FileSharing", "User", "UserID"),
    ("FileVersion", "User", "UploadedBy"),
    ("AccessRequest", "User", "UserId"),
    ("FeatureAccess", "User", "UserID"),
    ("ApplicationPermissionUser", "User", "UserID"),
    ("ReportDownload", "User", "UserID"),
    ("ApplicationRoleUser", "User", "UserID"),
    ("Group", "User", "Owner"),
    ("UserGroup", "User", "UserID"),
    ("ApplicationPermissionUser", "ApplicationPermission", "PermissionID"),
    ("ApplicationRolePermission", "ApplicationPermission", "PermissionID"),
    ("ApplicationRole", "Application", "ApplicationID"),
    ("FeatureAccess", "Application", "ApplicationID"),
    ("Notification", "Application", "ApplicationID"),
    ("ApplicationPropertyVal", "Application", "ApplicationID"),
    ("NotificationTopic", "Application", "ApplicationID"),
    ("ApplicationPermission", "Application", "ApplicationID"),
    ("ApplicationPropertyDef", "Application", "ApplicationID"),
    ("User", "Application", "InsertedFrom"),
]

# Crea un grafo vuoto
G = nx.DiGraph()  # Usa DiGraph per un grafo diretto

# Aggiungi i nodi e gli archi al grafo
for relazione in relazioni:
    G.add_edge(*relazione)

# Crea una disposizione per i nodi del grafo
pos = nx.spring_layout(G, k=0.5, iterations=20)
# pos = nx.shell_layout(G)
# pos = nx.spectral_layout(G)
# pos = nx.circular_layout(G)

# Crea un elenco di tutti i nodi
all_nodes = sorted(list(G.nodes()))

def draw_graph(nodes_to_show=all_nodes, show_incoming_edges=False, show_labels=True):
    # Se nodes_to_show è una stringa vuota, mostrare tutti i nodi, altrimenti filtrare
    if nodes_to_show == '':
        nodes = all_nodes
    else:
        nodes = []
        for node in nodes_to_show:
            ego_graph = nx.ego_graph(G if show_incoming_edges else nx.Graph(G), node)
            nodes.extend(ego_graph.nodes())
        nodes = list(set(nodes))  # rimuove i duplicati


    # Crea una mappa di colori per i nodi
    color_map = {node: plt.cm.Pastel2(i / len(nodes)) for i, node in enumerate(nodes)}

    # Grandezza grafico
    plt.rcParams["figure.figsize"] = [23, 9]

    # Disegna i nodi con colori diversi
    nx.draw_networkx_nodes(G, pos, nodelist=nodes, node_color=[color_map[node] for node in nodes])

    # Disegna gli archi con lo stesso colore del nodo di partenza
    for n1, n2 in G.edges():
        if n1 in nodes and n2 in nodes:
            nx.draw_networkx_edges(G, pos, edgelist=[(n1, n2)], edge_color=color_map[n1])

    # Disegna le etichette dei nodi se show_labels è True
    if show_labels:
        nx.draw_networkx_labels(G, pos, labels={node: node for node in nodes}, font_size=10)

    plt.show()

# Aggiungi un widget Checkbox e SelectMultiple
show_labels_check = Checkbox(value=True, description='Mostra etichette')
show_incoming_edges_check = Checkbox(value=False, description='NO collegamenti entrata')
show_nodes_SelectMultiple = SelectMultiple(options=all_nodes, value=all_nodes, rows=len(all_nodes),description='Nodi:')

# Usa la funzione interact per creare controlli interattivi
interact(draw_graph, nodes_to_show=show_nodes_SelectMultiple, show_incoming_edges=show_incoming_edges_check, show_labels=show_labels_check)

interactive(children=(SelectMultiple(description='Nodi:', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,…

<function __main__.draw_graph(nodes_to_show=['AccessRequest', 'Application', 'ApplicationPermission', 'ApplicationPermissionUser', 'ApplicationPropertyDef', 'ApplicationPropertyVal', 'ApplicationRole', 'ApplicationRoleApprover', 'ApplicationRolePermission', 'ApplicationRoleUser', 'Device', 'DeviceCategory', 'DeviceFamily', 'DevicePermission', 'DeviceRole', 'DeviceRolePermission', 'DeviceRole_User', 'Device_DeviceCategory', 'Feature', 'FeatureAccess', 'File', 'FileSharing', 'FileType', 'FileVersion', 'Group', 'Notification', 'NotificationTarget', 'NotificationTopic', 'ObjectId', 'ObjectIdsRequest', 'PhysicalDevice', 'PhysicalDevice_User', 'Relt', 'Report', 'ReportDownload', 'RoleRequest', 'RoleRequestApproval', 'User', 'UserGroup'], show_incoming_edges=False, show_labels=True, show_edge_labels=True)>